In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Visualizationdf_e.shapef
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score



In [2]:
# Load the dataset
df_n = pd.read_csv('../data/processed/cleaned_data_Namrata_NAN.csv')
df_e = pd.read_csv('../data/processed/cleaned_data_Emanuela.csv')



In [3]:
import warnings
warnings.filterwarnings("ignore")


In [4]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score



In [5]:
df_e.columns

Index(['sq_mt_built', 'n_rooms', 'n_bathrooms', 'is_exact_address_hidden',
       'floor', 'is_floor_under', 'rent_price', 'buy_price_by_area',
       'is_renewal_needed', 'is_new_development', 'has_central_heating',
       'has_individual_heating', 'has_ac', 'has_fitted_wardrobes', 'has_lift',
       'is_exterior', 'energy_certificate', 'has_parking',
       'is_orientation_north', 'is_orientation_west', 'is_orientation_south',
       'is_orientation_east', 'built_year', 'neighbourhood', 'district',
       'house_type'],
      dtype='object')

In [6]:
X = df_e[['sq_mt_built', 'n_rooms', 'n_bathrooms', 'is_exact_address_hidden',
       'floor', 'is_floor_under', 'rent_price', 'buy_price_by_area',
       'is_renewal_needed', 'is_new_development', 'has_central_heating',
       'has_individual_heating', 'has_ac', 'has_fitted_wardrobes', 'has_lift',
       'is_exterior', 'energy_certificate', 'has_parking',
       'is_orientation_north', 'is_orientation_west', 'is_orientation_south',
       'is_orientation_east', 'built_year', 'neighbourhood', 'district',
       'house_type']]

In [7]:
X.shape

(21517, 26)

In [8]:
X.isna().sum()

sq_mt_built                  126
n_rooms                        0
n_bathrooms                   16
is_exact_address_hidden        0
floor                       2594
is_floor_under              1163
rent_price                     0
buy_price_by_area              0
is_renewal_needed              0
is_new_development           976
has_central_heating         8082
has_individual_heating      8082
has_ac                         0
has_fitted_wardrobes           0
has_lift                    2368
is_exterior                 3018
energy_certificate             0
has_parking                    0
is_orientation_north       10294
is_orientation_west        10294
is_orientation_south       10294
is_orientation_east        10294
built_year                 11634
neighbourhood                  0
district                       0
house_type                     0
dtype: int64

In [9]:
X.dropna(
    inplace = True,
)

In [10]:
# Automatically identify numerical and categorical features
# Select numeric and categorical columns
numeric_cols = X.select_dtypes(include=[np.number]).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

# Define the preprocessor
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(), categorical_cols)
])


In [11]:

# Create and fit the KMeans model with preprocessing
def fit_kmeans(n_clusters, X):
    kmeans_pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("cluster", KMeans(n_clusters=n_clusters, random_state=9, verbose=0))
    ])
    kmeans_pipeline.fit(X)
    return kmeans_pipeline.named_steps["cluster"].inertia_

# Compute WCSS for different numbers of clusters
cluster_errors = []

for n_clusters in range(2, 11):
    wcsse = fit_kmeans(n_clusters,X)
    print('K = ', n_clusters, '\tWCSS Err. = ', wcsse)
    cluster_errors.append(wcsse)

# Plot the SSE for different numbers of clusters
plt.plot(range(2, 11), cluster_errors, "o-")
plt.xlabel("No. Clusters")
plt.ylabel("SSE")
plt.title("Elbow Method")
plt.show()


K =  2 	WCSS Err. =  101328.05407164531
K =  3 	WCSS Err. =  94350.17210821633
K =  4 	WCSS Err. =  90353.5347686433
K =  5 	WCSS Err. =  87132.33410171521
K =  6 	WCSS Err. =  83175.15263781919
K =  7 	WCSS Err. =  81322.26349309672
K =  8 	WCSS Err. =  78923.40799479173
K =  9 	WCSS Err. =  77542.16966011061
K =  10 	WCSS Err. =  75662.23448928089


In [12]:
from sklearn.metrics import silhouette_samples, silhouette_score


silhouette_s = []

for n_clusters in range(2, 11):
    kmeans_pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("cluster", KMeans(n_clusters=n_clusters, random_state=9, verbose=0))
    ])

    # Fit the pipeline and get the cluster labels
    cluster_labels = kmeans_pipeline.fit_predict(X)
    
    # Get the preprocessed data
    X_tr = kmeans_pipeline.named_steps["preprocessor"].transform(X)
    
    silhouette_avg = silhouette_score(X_tr, cluster_labels).round(4)
    print(f"For n_clusters = {n_clusters}, The average silhouette_score is : {silhouette_avg}")
    
    silhouette_s.append(silhouette_avg)

# Plot the Silhouette Scores for different numbers of clusters
plt.plot(range(2, 11), silhouette_s, "o-")
plt.xlabel("No. Clusters")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Scores for Different Numbers of Clusters")
plt.show()

For n_clusters = 2, The average silhouette_score is : 0.1253
For n_clusters = 3, The average silhouette_score is : 0.0944
For n_clusters = 4, The average silhouette_score is : 0.1003
For n_clusters = 5, The average silhouette_score is : 0.1044
For n_clusters = 6, The average silhouette_score is : 0.0988
For n_clusters = 7, The average silhouette_score is : 0.0853
For n_clusters = 8, The average silhouette_score is : 0.0872
For n_clusters = 9, The average silhouette_score is : 0.076
For n_clusters = 10, The average silhouette_score is : 0.0775


<pre>

| Range       | Interpretation                                |
|-------------|-----------------------------------------------|
| 0.71 - 1.0  | A strong structure has been found.            |
| 0.51 - 0.7  | A reasonable structure has been found.        |
| 0.26 - 0.5  | The structure is weak and could be artificial.|
| < 0.25      | No substantial structure has been found.      |

</pre>


__2 - 3 Clusters__

In [26]:
# Create an 'age' column
current_year = 2024
X['age'] = current_year - X['built_year']

numeric_cols = X.select_dtypes(include=[np.number]).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

# Define the preprocessor
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(), categorical_cols)
])

# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('kmeans', KMeans(n_clusters=3, random_state=42))
])


# Fit the pipeline
pipeline.fit(X)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['sq_mt_built', 'n_rooms', 'n_bathrooms', 'is_exact_address_hidden',
       'is_floor_under', 'rent_price', 'buy_price_by_area',
       'is_renewal_needed', 'is_new_development', 'has_central_heating',
       'has_individual_heating', 'has_ac', 'has_fitted_wardrobes', 'has_lift',
       'is_exterior', 'has_parking', 'is_orientation_north',
       'is_orientation_west', 'is_orientation_south', 'is_orientation_east',
       'built_year', 'age', 'age_cluster'],
      dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index(['floor', 'energy_certificate', 'neighbourhood', 'district',
       'house_type'],
      dtype='object'))])),
                ('kmeans', KMeans(n_clusters=3, random_state=42))])

In [27]:
# Get the cluster labels

X['age_cluster'] = pipeline.named_steps['kmeans'].labels_



In [28]:
# Analyze clusters
#print(X.groupby('age_cluster').mean())

# Visualization (Optional)
sns.scatterplot(x='age', y='buy_price_by_area', hue='age_cluster', data=X)
plt.show()